# Read Cosmos OLAP Store
## Prerequiste

1. Go to your Synapse Analytics workspace.
    1. Create a Linked Data connection for your SQL API account.
    2. Under the Data blade, select the **+ (plus)** sign.
    3. Select the **Connect to external data** option.
    4. Now select the **Azure Cosmos DB (SQLAPI)** option.
    5. Enter all the information regarding your specific Azure Cosmos DB account either by using the dropdowns or by entering the connection string.
    6. Take note of the name you assigned to your Linked Data connection.
    1. Use the Linked Data connection name to replace {Linked Synapse DB Name} placeholders in Step #1 and Step #7 below.
  
2. Test the connection by looking for your database accounts in the Data blade, and under the Linked tab.
    - There should be a list that contains all accounts and collections.
    - Collections that have an Analytical Store enabled will have a distinctive icon.

## Step 1: Load Data Frame

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

# Read from Cosmos DB analytical store into a Spark DataFrame and display 5 rows from the DataFrame
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "{Linked Synapse DB Name}")\
    .option("spark.cosmos.container", "CallRecords")\
    .load()\
   
    
display (df.limit(5))

## Step 2: View Schema Representation


The default option for Azure Cosmos DB CORE (SQL) API, is **Well defined Schema**, but we have used **Full Fidelity Schema** . For more information about schemas representation, click [here](https://docs.microsoft.com/azure/cosmos-db/analytical-store-introduction#schema-representation) .

In [ ]:
df.printSchema()

## Step 3: Unpack the Dataframe
For aggregation, a syntax that doesn't explicity mention the datatypes executes without an error. But it is **not recommended!**

It runs because Spark automatically flattens the structure into an Array, where it takes each distinct value in the struct dict and applies the aggregation function.

However, when we have more than one datatype for the same struct of a property, the implicit conversion by Spark does can cause wrong results.

The below code we will:
1. Select just the pertinent columns.
2. Unpack the struct columns with df.attrname.datatype syntax.
3. Rename the columns of the unpacked dataframe
4. Cast the DateTime fields from string to timestamp (ISO 8601 and other formats will be read by Synapse as a string datatype. Hence you need to use regular functions to cast it to datetime).
5. Filter the dataframe to limit data to JUN2022 BillCycle

In [ ]:
# Select just the pertinent columns and unpack with df.attrname.datatype syntax

df_unpacked = df.select(
    df.StartDateTime.string,
    df.EndDateTime.string,
    df.DurationSec.num,
    df.CallFrom.string,
    df.CallTo.string, 
    df.CallType.string,
    df.CallLocationId.num,
    df.BaseLocationId.num,
    df.IsRoaming.bool,
    df.IsIncoming.bool,
    df.SubscriberId.string,
    df.BillCycle.string,
    df.pk.string
    )

display (df_unpacked.limit(5))

# Rename the columns of the unpacked DataFrame
new_column_names = [
    'StartDateTime', 'EndDateTime', 'DurationSec', 'CallFrom', 'CallTo','CallType','CallLocationId','BaseLocationId','IsRoaming','IsIncoming','SubscriberId','BillCycle','PK']
df_renamed= df_unpacked.toDF(*new_column_names)

# cast string to timestamp
from pyspark.sql.types import TimestampType
df_cast=df_renamed.withColumn("StartDateTime",df_renamed["StartDateTime"].cast(TimestampType()))
df_cast=df_cast.withColumn("EndDateTime",df_cast["EndDateTime"].cast(TimestampType()))

displayHTML("After Renaming Columns and Cast")
display (df_cast.limit(5))

# Filter the dataframe to limit data to current Billcycle
df_flat= df_cast.filter(col('BillCycle')=="JUN2022")


## Step 4: Converting Flat Dataframe to Nested Dataframe
Copy the call details into a new column called "Log"

In [ ]:
df_nested= df_flat.withColumn(
  "Log",
  F.struct(
    F.col("StartDateTime").alias("StartDateTime"),
    F.col("EndDateTime").alias("EndDateTime"),
    F.col("DurationSec").alias("DurationSec"),
    F.col("CallFrom").alias("CallFrom"),
    F.col("CallTo").alias("CallTo"),
    F.col("CallType").alias("CallType")
  )
)

display (df_nested.limit(5))

## Step 5: Calculate Cost

The code below emulates the bill generation using a hyper simplistic algorithm
1. Conditonally calculate cost based on 'IsIncoming' column value


In [ ]:
#Conditional cost calculation based on 'IsIncoming' column value 
df_calculate=df_nested.withColumn('Cost',
              F.when(F.col('IsIncoming') == 0, F.col('DurationSec') * 0.005).otherwise(
                F.when(F.col('IsIncoming') == 1, F.col('DurationSec') * 0.001)))


display (df_calculate.limit(5))

## Step 6: Aggregate Data for Bill Generation

In [ ]:
# group by SubscriberId and PK and  then total the Cost
df_agg=df_calculate.groupby("SubscriberId","PK").agg(F.collect_list("Log"), F.sum("Cost"))

# rename the columns
new_column_names = [
    'SubscriberId','id', 'Log', 'Cost']
df_write= df_agg.toDF(*new_column_names)

#reorder the columns
df_write=df_write.select( 'id','SubscriberId', 'Cost','Log')

display(df_write.limit(5))


## Step 7: Insert data into Cosmos OLTP Store

Use dataframe .save to insert the data frame into the Cosmos DB OLTP store

In [ ]:
df_write.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "{Linked Synapse DB Name}")\
    .option("spark.cosmos.container", "Bills")\
    .mode('append')\
    .save()